### DCGAN's (Deep Convlutional Generative Adversarial Networks).

Generative Adversarial Networks `(GANs)` are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A **generator** ("the artist") learns to create images that look real, while a **discriminator** ("the art critic") learns to tell real images apart from fakes.


````

       [-------------]                  [--------------]
   --> [Discriminator] -> (fake/real)   [  Generator   ]-|
   |   [_____________]                  [______________] |
   |_____________________________________________________|
                       (Generated image)(fake)

````
1. So the generator its purpose is to genarate images from random noise and pass to he discriminator to detemine if the image is real or fake.
2. The discriminator then detemines if the image is for sure real or fake.
3. So we have 2 agents in an environment (discriminator) and (generator)
4. Their goal is to maximize their rewards. **But how?**.
5. A generator gets it's reward when a discriminator fake fake from real images.
6. A discrimnator then get's it's reward if it can not be fooled by the generator.
7. **As we train more and more the generator model will become better and better so that it will be able to produce fake images that a discriminator model can not discriminate from real images.**

### Let's implement this.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image , image_dataset_from_directory
import os

### Renaming all the images.

In [ ]:
# for i, image in enumerate(os.listdir('./data/images/')):
#     os.rename(f'data/images/{image}', f'data/images/img_{i}.jpg')
# print("Done")

### Folder Structures

```
data
    images
```

### Creating a dataset

In [ ]:
ds = image_dataset_from_directory(
    './data',
    label_mode = None,# because unsupervised
    color_mode='rgb',
    batch_size=32,
    image_size=(64, 64),
).map(lambda x: x/255)

### Displaying some images

In [ ]:
for x in ds:
    plt.axis("off")
    plt.imshow((x.numpy() * 255).astype("int32")[0], cmap="gray")
    break

### Create the discriminator.
This model will maps the (`image_width` x `image_height`) in our case (`64` x `64`) to a binary class (**fake** or **real**)

In [ ]:
discriminator = keras.Sequential([
    keras.layers.Input(shape=(64, 64, 3)),
    keras.layers.Conv2D(64, kernel_size=(4, 4), padding="same", strides=(2, 2)),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=(4, 4), padding="same", strides=(2, 2)),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=(4, 4), padding="same", strides=(2, 2)),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Flatten(),
    keras.layers.Dropout(.2),
    keras.layers.Dense(1, activation="sigmoid"),
], name="discriminator")

discriminator.summary()

### Create the generator.
This model is the mirror of the `discriminator` model. Replacing `Conv2D` layers with `Conv2DTranspose` layers.

* `Conv2DTranspose` - is just an inverse of `Conv2D`.
* Our goal is to create an image from a vector to an image of shape `(64, 64, 3)`.
* We are going to have `128` latent_dim.
* `"latent_dim"` is the number of nodes used as input of the generator.

#### Implementation
1. We will start with the image of shape `8` x `8`.
2. Add a reshape layer to get shape (8 x 8 x latent_dim)
3. Our image now is in a shape that we can put in a `Conv` layer.
4. We will then add a `Conv2DTranspose` layer with `strides =2`. Why? Remember in a normal Conv2D if we have an image of shape (16, 16)  and strides are (2, 2) the image will be reduced to (8, 8) but in a `Conv2DTranspose` the image will be increased to (32, 32) shape, which is our goal. So we want to add these `Conv2DTranspose` layers until we reach (64, 64) image shape.

5. We will then add `Conv2D` layer with 3 input features, this is because we want to output 3 channels.


In [ ]:
latent_dim = 128
generator = keras.Sequential([
    keras.layers.Input(shape=(latent_dim, )),
    keras.layers.Dense(8 * 8 * latent_dim, activation="relu"),
    keras.layers.Reshape((8, 8, latent_dim)),
    keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
], name="generator")

generator.summary()

### Override `train_step`

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        
    # .compile()
    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")
        
    @property
    def metric(self):
        return [
            self.d_loss_metric,
            self.g_loss_metric
        ]
    
    # .fit()
    def train_step(self, real_images):
        # Sample random points in the latent space
        """
        We need to create latent_vectors with shape (batch_size_of_real_images, latent_dim)
        """
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        """
        Generate fake images of size (batch_size)
        """
        generated_images = self.generator(random_latent_vectors)
        
        """
        Combine images, generated and real
        """
        combined_images = tf.concat([generated_images, real_images], axis=0)

        """
        Assemble labels discriminating real from fake images
        """
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        """
        Add random noise to the labels - important trick!
        """
        labels += 0.05 * tf.random.uniform(tf.shape(labels))
        
        """
        Train the discriminator.
        """
        
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
            
        gradients = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(gradients, self.discriminator.trainable_weights )
        )
        
        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))
        """
        Train the generator (note that we should *not* update the weights
        of the discriminator)!
        """
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

### Create a callback that periodically saves `generated` images

In [ ]:
import os
if not os.path.exists('generated'):
    os.mkdir('generated')

class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated/generated_img_%03d_%d.png" % (epoch, i))

### Train the end-to-end model.

In [ ]:
epochs = 10  # In practice, use ~100 epochs
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)
gan.fit(
    ds, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

> If we train the model above for longer on a `GPU` we will be able to achive reasonable results.

### Reference.

* [Keras DCGANs](https://keras.io/examples/generative/dcgan_overriding_train_step/)